**Project 4 Notebook 2**

In which I decide to go back to using all lowercase, and tokenize into ngrams before removing stopwords.

At the end of the notebook, I decide to tokenize by single words only. 

In [1]:
import pandas as pd
import numpy as np
import re
from pprint import pprint
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
#warnings.filterwarnings("ignore",category=DeprecationWarning)

Read the data files, make the two DataFrames and append them together.

In [2]:
data = pd.read_csv('Project-4-data/fanfiction-katniss1_pre_page_69.csv')
data.head()

,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text,previous_pages,previous_pages-href
0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...","For the Tumblr Everlark fic exchange, Spring e...",« Prev,https://www.fanfiction.net/book/Hunger-Games/?...
1,1613674516-43761,https://www.fanfiction.net/book/Hunger-Games/?...,Reading The Signs,https://www.fanfiction.net/s/11032211/1/Readin...,Reading The Signs,notalone91,https://www.fanfiction.net/u/2695835/notalone91,Rated: Fiction T - English - Angst/Drama - Kat...,A/N: Hey all. It's been ages since I posted a ...,« Prev,https://www.fanfiction.net/book/Hunger-Games/?...
2,1613674143-43610,https://www.fanfiction.net/book/Hunger-Games/?...,A Chronicle of Lies,https://www.fanfiction.net/s/11535436/1/A-Chro...,A Chronicle of Lies,whitelilly0989,https://www.fanfiction.net/u/1296268/whitelill...,Rated: Fiction T - English - Drama - Katniss E...,Author NotesI wrote this a long time ago. This...,« Prev,https://www.fanfiction.net/book/Hunger-Games/?...
3,1613675615-44227,https://www.fanfiction.net/book/Hunger-Games/?...,Watch Me Fall Apart,https://www.fanfiction.net/s/10203191/1/Watch-...,Watch Me Fall Apart,somethingtobelieve,https://www.fanfiction.net/u/5581135/something...,Rated: Fiction T - English - Angst/Romance - K...,"Written for Prompts in Panem, Day 2 - Marigold...",« Prev,https://www.fanfiction.net/book/Hunger-Games/?...
4,1613675915-44351,https://www.fanfiction.net/book/Hunger-Games/?...,100 Channels and Nothing On,https://www.fanfiction.net/s/9999075/1/100-Cha...,100 Channels and Nothing On,Izzy Samson,https://www.fanfiction.net/u/4073375/Izzy-Samson,Rated: Fiction T - English - Hurt/Comfort - Ka...,Thanks to my wonderful betas Court81981 and Ki...,« Prev,https://www.fanfiction.net/book/Hunger-Games/?...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1725 entries, 0 to 1724
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   web-scraper-order      1725 non-null   object
 1   web-scraper-start-url  1725 non-null   object
 2   story_link             1725 non-null   object
 3   story_link-href        1725 non-null   object
 4   story_title            1725 non-null   object
 5   author_id              1725 non-null   object
 6   author_id-href         1725 non-null   object
 7   story_info             1725 non-null   object
 8   story_text             1725 non-null   object
 9   previous_pages         1700 non-null   object
 10  previous_pages-href    1700 non-null   object
dtypes: object(11)
memory usage: 148.4+ KB


In [4]:
data2=pd.read_csv('Project-4-data/fanfiction-katniss1_p69-end_complete.csv')
data.head()

,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text,previous_pages,previous_pages-href
0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...","For the Tumblr Everlark fic exchange, Spring e...",« Prev,https://www.fanfiction.net/book/Hunger-Games/?...
1,1613674516-43761,https://www.fanfiction.net/book/Hunger-Games/?...,Reading The Signs,https://www.fanfiction.net/s/11032211/1/Readin...,Reading The Signs,notalone91,https://www.fanfiction.net/u/2695835/notalone91,Rated: Fiction T - English - Angst/Drama - Kat...,A/N: Hey all. It's been ages since I posted a ...,« Prev,https://www.fanfiction.net/book/Hunger-Games/?...
2,1613674143-43610,https://www.fanfiction.net/book/Hunger-Games/?...,A Chronicle of Lies,https://www.fanfiction.net/s/11535436/1/A-Chro...,A Chronicle of Lies,whitelilly0989,https://www.fanfiction.net/u/1296268/whitelill...,Rated: Fiction T - English - Drama - Katniss E...,Author NotesI wrote this a long time ago. This...,« Prev,https://www.fanfiction.net/book/Hunger-Games/?...
3,1613675615-44227,https://www.fanfiction.net/book/Hunger-Games/?...,Watch Me Fall Apart,https://www.fanfiction.net/s/10203191/1/Watch-...,Watch Me Fall Apart,somethingtobelieve,https://www.fanfiction.net/u/5581135/something...,Rated: Fiction T - English - Angst/Romance - K...,"Written for Prompts in Panem, Day 2 - Marigold...",« Prev,https://www.fanfiction.net/book/Hunger-Games/?...
4,1613675915-44351,https://www.fanfiction.net/book/Hunger-Games/?...,100 Channels and Nothing On,https://www.fanfiction.net/s/9999075/1/100-Cha...,100 Channels and Nothing On,Izzy Samson,https://www.fanfiction.net/u/4073375/Izzy-Samson,Rated: Fiction T - English - Hurt/Comfort - Ka...,Thanks to my wonderful betas Court81981 and Ki...,« Prev,https://www.fanfiction.net/book/Hunger-Games/?...


In [5]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1718 entries, 0 to 1717
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   web-scraper-order      1718 non-null   object
 1   web-scraper-start-url  1718 non-null   object
 2   story_link             1718 non-null   object
 3   story_link-href        1718 non-null   object
 4   story_title            1718 non-null   object
 5   author_id              1718 non-null   object
 6   author_id-href         1718 non-null   object
 7   story_info             1718 non-null   object
 8   story_text             1718 non-null   object
 9   next_pages             1693 non-null   object
 10  next_pages-href        1693 non-null   object
dtypes: object(11)
memory usage: 147.8+ KB


Append the dataframes to make a dataframe with the complete dataset.

In [6]:
katniss=data.append(data2)
katniss.head()

,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text,previous_pages,previous_pages-href,next_pages,next_pages-href
0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...","For the Tumblr Everlark fic exchange, Spring e...",« Prev,https://www.fanfiction.net/book/Hunger-Games/?...,NaN,NaN
1,1613674516-43761,https://www.fanfiction.net/book/Hunger-Games/?...,Reading The Signs,https://www.fanfiction.net/s/11032211/1/Readin...,Reading The Signs,notalone91,https://www.fanfiction.net/u/2695835/notalone91,Rated: Fiction T - English - Angst/Drama - Kat...,A/N: Hey all. It's been ages since I posted a ...,« Prev,https://www.fanfiction.net/book/Hunger-Games/?...,NaN,NaN
2,1613674143-43610,https://www.fanfiction.net/book/Hunger-Games/?...,A Chronicle of Lies,https://www.fanfiction.net/s/11535436/1/A-Chro...,A Chronicle of Lies,whitelilly0989,https://www.fanfiction.net/u/1296268/whitelill...,Rated: Fiction T - English - Drama - Katniss E...,Author NotesI wrote this a long time ago. This...,« Prev,https://www.fanfiction.net/book/Hunger-Games/?...,NaN,NaN
3,1613675615-44227,https://www.fanfiction.net/book/Hunger-Games/?...,Watch Me Fall Apart,https://www.fanfiction.net/s/10203191/1/Watch-...,Watch Me Fall Apart,somethingtobelieve,https://www.fanfiction.net/u/5581135/something...,Rated: Fiction T - English - Angst/Romance - K...,"Written for Prompts in Panem, Day 2 - Marigold...",« Prev,https://www.fanfiction.net/book/Hunger-Games/?...,NaN,NaN
4,1613675915-44351,https://www.fanfiction.net/book/Hunger-Games/?...,100 Channels and Nothing On,https://www.fanfiction.net/s/9999075/1/100-Cha...,100 Channels and Nothing On,Izzy Samson,https://www.fanfiction.net/u/4073375/Izzy-Samson,Rated: Fiction T - English - Hurt/Comfort - Ka...,Thanks to my wonderful betas Court81981 and Ki...,« Prev,https://www.fanfiction.net/book/Hunger-Games/?...,NaN,NaN


In [7]:
katniss=data.append(data2)
katniss.head()

,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text,previous_pages,previous_pages-href,next_pages,next_pages-href
0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...","For the Tumblr Everlark fic exchange, Spring e...",« Prev,https://www.fanfiction.net/book/Hunger-Games/?...,NaN,NaN
1,1613674516-43761,https://www.fanfiction.net/book/Hunger-Games/?...,Reading The Signs,https://www.fanfiction.net/s/11032211/1/Readin...,Reading The Signs,notalone91,https://www.fanfiction.net/u/2695835/notalone91,Rated: Fiction T - English - Angst/Drama - Kat...,A/N: Hey all. It's been ages since I posted a ...,« Prev,https://www.fanfiction.net/book/Hunger-Games/?...,NaN,NaN
2,1613674143-43610,https://www.fanfiction.net/book/Hunger-Games/?...,A Chronicle of Lies,https://www.fanfiction.net/s/11535436/1/A-Chro...,A Chronicle of Lies,whitelilly0989,https://www.fanfiction.net/u/1296268/whitelill...,Rated: Fiction T - English - Drama - Katniss E...,Author NotesI wrote this a long time ago. This...,« Prev,https://www.fanfiction.net/book/Hunger-Games/?...,NaN,NaN
3,1613675615-44227,https://www.fanfiction.net/book/Hunger-Games/?...,Watch Me Fall Apart,https://www.fanfiction.net/s/10203191/1/Watch-...,Watch Me Fall Apart,somethingtobelieve,https://www.fanfiction.net/u/5581135/something...,Rated: Fiction T - English - Angst/Romance - K...,"Written for Prompts in Panem, Day 2 - Marigold...",« Prev,https://www.fanfiction.net/book/Hunger-Games/?...,NaN,NaN
4,1613675915-44351,https://www.fanfiction.net/book/Hunger-Games/?...,100 Channels and Nothing On,https://www.fanfiction.net/s/9999075/1/100-Cha...,100 Channels and Nothing On,Izzy Samson,https://www.fanfiction.net/u/4073375/Izzy-Samson,Rated: Fiction T - English - Hurt/Comfort - Ka...,Thanks to my wonderful betas Court81981 and Ki...,« Prev,https://www.fanfiction.net/book/Hunger-Games/?...,NaN,NaN


In [8]:
##Can delete columns "previous_pages" and "next_pages". 
##These are links that the scraping extension put in.
katniss.drop(["previous_pages", "previous_pages-href",
             "next_pages", "next_pages-href"], axis=1, inplace=True )

In [9]:
katniss.head()

,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text
0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...","For the Tumblr Everlark fic exchange, Spring e..."
1,1613674516-43761,https://www.fanfiction.net/book/Hunger-Games/?...,Reading The Signs,https://www.fanfiction.net/s/11032211/1/Readin...,Reading The Signs,notalone91,https://www.fanfiction.net/u/2695835/notalone91,Rated: Fiction T - English - Angst/Drama - Kat...,A/N: Hey all. It's been ages since I posted a ...
2,1613674143-43610,https://www.fanfiction.net/book/Hunger-Games/?...,A Chronicle of Lies,https://www.fanfiction.net/s/11535436/1/A-Chro...,A Chronicle of Lies,whitelilly0989,https://www.fanfiction.net/u/1296268/whitelill...,Rated: Fiction T - English - Drama - Katniss E...,Author NotesI wrote this a long time ago. This...
3,1613675615-44227,https://www.fanfiction.net/book/Hunger-Games/?...,Watch Me Fall Apart,https://www.fanfiction.net/s/10203191/1/Watch-...,Watch Me Fall Apart,somethingtobelieve,https://www.fanfiction.net/u/5581135/something...,Rated: Fiction T - English - Angst/Romance - K...,"Written for Prompts in Panem, Day 2 - Marigold..."
4,1613675915-44351,https://www.fanfiction.net/book/Hunger-Games/?...,100 Channels and Nothing On,https://www.fanfiction.net/s/9999075/1/100-Cha...,100 Channels and Nothing On,Izzy Samson,https://www.fanfiction.net/u/4073375/Izzy-Samson,Rated: Fiction T - English - Hurt/Comfort - Ka...,Thanks to my wonderful betas Court81981 and Ki...


In [10]:
katniss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3443 entries, 0 to 1717
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   web-scraper-order      3443 non-null   object
 1   web-scraper-start-url  3443 non-null   object
 2   story_link             3443 non-null   object
 3   story_link-href        3443 non-null   object
 4   story_title            3443 non-null   object
 5   author_id              3443 non-null   object
 6   author_id-href         3443 non-null   object
 7   story_info             3443 non-null   object
 8   story_text             3443 non-null   object
dtypes: object(9)
memory usage: 269.0+ KB


In [11]:
katniss.to_csv('katniss_complete_clean_restart_2_24.csv')

In [13]:
#replace punctuation with a white space, remove numbers, capital letters
##on 2/23, decided to not replace capital letters
##on 2/24, decided to go back and replace capital letters with lowercase, and then not tag parts of speech, as the pos
##tagger will not recognize some names as nouns (eg Katniss, Peeta, Haymitch). Captialized stopwords
##were not being removed, which creates its own mess.
import re
import string

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower()) #this was used 2/22 to replace
#capital letters and remove punctuation.
#punc_remove = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x) this is from 2/23
katniss['story_text_lower_nonumpunc'] = data.story_text.map(alphanumeric).map(punc_lower)
katniss.head()

,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text,story_text_lower_nonumpunc
0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...","For the Tumblr Everlark fic exchange, Spring e...",for the tumblr everlark fic exchange spring e...
1,1613674516-43761,https://www.fanfiction.net/book/Hunger-Games/?...,Reading The Signs,https://www.fanfiction.net/s/11032211/1/Readin...,Reading The Signs,notalone91,https://www.fanfiction.net/u/2695835/notalone91,Rated: Fiction T - English - Angst/Drama - Kat...,A/N: Hey all. It's been ages since I posted a ...,a n hey all it s been ages since i posted a ...
2,1613674143-43610,https://www.fanfiction.net/book/Hunger-Games/?...,A Chronicle of Lies,https://www.fanfiction.net/s/11535436/1/A-Chro...,A Chronicle of Lies,whitelilly0989,https://www.fanfiction.net/u/1296268/whitelill...,Rated: Fiction T - English - Drama - Katniss E...,Author NotesI wrote this a long time ago. This...,author notesi wrote this a long time ago this...
3,1613675615-44227,https://www.fanfiction.net/book/Hunger-Games/?...,Watch Me Fall Apart,https://www.fanfiction.net/s/10203191/1/Watch-...,Watch Me Fall Apart,somethingtobelieve,https://www.fanfiction.net/u/5581135/something...,Rated: Fiction T - English - Angst/Romance - K...,"Written for Prompts in Panem, Day 2 - Marigold...",written for prompts in panem day marigold...
4,1613675915-44351,https://www.fanfiction.net/book/Hunger-Games/?...,100 Channels and Nothing On,https://www.fanfiction.net/s/9999075/1/100-Cha...,100 Channels and Nothing On,Izzy Samson,https://www.fanfiction.net/u/4073375/Izzy-Samson,Rated: Fiction T - English - Hurt/Comfort - Ka...,Thanks to my wonderful betas Court81981 and Ki...,thanks to my wonderful betas and written ...


In [14]:
katniss.to_csv('katniss_complete_restart_nonumpuncUC_2_24.csv')

In [15]:
#Drop the column "story_text" that still has numbers, capital letters, etc.
katniss.drop(["story_text"], axis=1, inplace=True )

In [16]:
katniss.head()

,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_lower_nonumpunc
0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...",for the tumblr everlark fic exchange spring e...
1,1613674516-43761,https://www.fanfiction.net/book/Hunger-Games/?...,Reading The Signs,https://www.fanfiction.net/s/11032211/1/Readin...,Reading The Signs,notalone91,https://www.fanfiction.net/u/2695835/notalone91,Rated: Fiction T - English - Angst/Drama - Kat...,a n hey all it s been ages since i posted a ...
2,1613674143-43610,https://www.fanfiction.net/book/Hunger-Games/?...,A Chronicle of Lies,https://www.fanfiction.net/s/11535436/1/A-Chro...,A Chronicle of Lies,whitelilly0989,https://www.fanfiction.net/u/1296268/whitelill...,Rated: Fiction T - English - Drama - Katniss E...,author notesi wrote this a long time ago this...
3,1613675615-44227,https://www.fanfiction.net/book/Hunger-Games/?...,Watch Me Fall Apart,https://www.fanfiction.net/s/10203191/1/Watch-...,Watch Me Fall Apart,somethingtobelieve,https://www.fanfiction.net/u/5581135/something...,Rated: Fiction T - English - Angst/Romance - K...,written for prompts in panem day marigold...
4,1613675915-44351,https://www.fanfiction.net/book/Hunger-Games/?...,100 Channels and Nothing On,https://www.fanfiction.net/s/9999075/1/100-Cha...,100 Channels and Nothing On,Izzy Samson,https://www.fanfiction.net/u/4073375/Izzy-Samson,Rated: Fiction T - English - Hurt/Comfort - Ka...,thanks to my wonderful betas and written ...


I have story texts all lowercase, with numbers and punctuation gone. Next step is to tokenize into single words, and then into ngrams.

In [19]:
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
import re
#regular expression library
import string
from nltk.corpus import stopwords
#set(stopwords.words('english'))

In [25]:
##Tokenize by single words
katniss['story_text_wtokenized'] = katniss['story_text_lower_nonumpunc'].apply(word_tokenize)
katniss.head()

,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_lower_nonumpunc,story_text_wtokenized
0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...",for the tumblr everlark fic exchange spring e...,"[for, the, tumblr, everlark, fic, exchange, sp..."
1,1613674516-43761,https://www.fanfiction.net/book/Hunger-Games/?...,Reading The Signs,https://www.fanfiction.net/s/11032211/1/Readin...,Reading The Signs,notalone91,https://www.fanfiction.net/u/2695835/notalone91,Rated: Fiction T - English - Angst/Drama - Kat...,a n hey all it s been ages since i posted a ...,"[a, n, hey, all, it, s, been, ages, since, i, ..."
2,1613674143-43610,https://www.fanfiction.net/book/Hunger-Games/?...,A Chronicle of Lies,https://www.fanfiction.net/s/11535436/1/A-Chro...,A Chronicle of Lies,whitelilly0989,https://www.fanfiction.net/u/1296268/whitelill...,Rated: Fiction T - English - Drama - Katniss E...,author notesi wrote this a long time ago this...,"[author, notesi, wrote, this, a, long, time, a..."
3,1613675615-44227,https://www.fanfiction.net/book/Hunger-Games/?...,Watch Me Fall Apart,https://www.fanfiction.net/s/10203191/1/Watch-...,Watch Me Fall Apart,somethingtobelieve,https://www.fanfiction.net/u/5581135/something...,Rated: Fiction T - English - Angst/Romance - K...,written for prompts in panem day marigold...,"[written, for, prompts, in, panem, day, marigo..."
4,1613675915-44351,https://www.fanfiction.net/book/Hunger-Games/?...,100 Channels and Nothing On,https://www.fanfiction.net/s/9999075/1/100-Cha...,100 Channels and Nothing On,Izzy Samson,https://www.fanfiction.net/u/4073375/Izzy-Samson,Rated: Fiction T - English - Hurt/Comfort - Ka...,thanks to my wonderful betas and written ...,"[thanks, to, my, wonderful, betas, and, writte..."


In [26]:
katniss.drop(["story_text_lower_nonumpunc"], axis=1, inplace=True )

In [27]:
katniss.head()

,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_wtokenized
0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...","[for, the, tumblr, everlark, fic, exchange, sp..."
1,1613674516-43761,https://www.fanfiction.net/book/Hunger-Games/?...,Reading The Signs,https://www.fanfiction.net/s/11032211/1/Readin...,Reading The Signs,notalone91,https://www.fanfiction.net/u/2695835/notalone91,Rated: Fiction T - English - Angst/Drama - Kat...,"[a, n, hey, all, it, s, been, ages, since, i, ..."
2,1613674143-43610,https://www.fanfiction.net/book/Hunger-Games/?...,A Chronicle of Lies,https://www.fanfiction.net/s/11535436/1/A-Chro...,A Chronicle of Lies,whitelilly0989,https://www.fanfiction.net/u/1296268/whitelill...,Rated: Fiction T - English - Drama - Katniss E...,"[author, notesi, wrote, this, a, long, time, a..."
3,1613675615-44227,https://www.fanfiction.net/book/Hunger-Games/?...,Watch Me Fall Apart,https://www.fanfiction.net/s/10203191/1/Watch-...,Watch Me Fall Apart,somethingtobelieve,https://www.fanfiction.net/u/5581135/something...,Rated: Fiction T - English - Angst/Romance - K...,"[written, for, prompts, in, panem, day, marigo..."
4,1613675915-44351,https://www.fanfiction.net/book/Hunger-Games/?...,100 Channels and Nothing On,https://www.fanfiction.net/s/9999075/1/100-Cha...,100 Channels and Nothing On,Izzy Samson,https://www.fanfiction.net/u/4073375/Izzy-Samson,Rated: Fiction T - English - Hurt/Comfort - Ka...,"[thanks, to, my, wonderful, betas, and, writte..."


In [28]:
katniss.to_csv('katniss_complete_restart_wtokenized_2_24.csv')

I tried to make quadgrams, but could not remove the stopwords.

In [50]:
#Make quadgrams from each individual story in the 'story_text_wtokenized' column of the dataframe. Try to
#put this into the dataframe as another column.
katniss_quadgrams_list=[]
for story in katniss['story_text_wtokenized']:
    quadgrams = list(ngrams(story, 4))
    katniss_quadgrams_list.append(quadgrams)
katniss['story_text_quadgrams']=katniss_quadgrams_list

In [51]:
katniss.head(1)

,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_wtokenized,story_text_quadgrams
0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...","[for, the, tumblr, everlark, fic, exchange, sp...","[(for, the, tumblr, everlark), (the, tumblr, e..."


In [52]:
katniss.to_csv('katniss_complete_restart_quadgrams_2_24.csv')

In [31]:
#Make quadgrams from each individual story in the 'story_text_wtokenized' column of the dataframe.
#This for loop makes a list, with each story being a list of tuples, each tuple is a quadgram.
katniss_quadgrams_list=[]
for story in katniss['story_text_wtokenized']:
    quadgrams = list(ngrams(story, 4))
    katniss_quadgrams_list.append(quadgrams)

In [37]:
print(katniss_quadgrams_list[3000])

[('written', 'from', 'a', 'kind'), ('from', 'a', 'kind', 'of'), ('a', 'kind', 'of', 'darker'), ('kind', 'of', 'darker', 'perspective'), ('of', 'darker', 'perspective', 'peeta'), ('darker', 'perspective', 'peeta', 'seems'), ('perspective', 'peeta', 'seems', 'a'), ('peeta', 'seems', 'a', 'little'), ('seems', 'a', 'little', 'more'), ('a', 'little', 'more', 'broken'), ('little', 'more', 'broken', 'in'), ('more', 'broken', 'in', 'this'), ('broken', 'in', 'this', 'story'), ('in', 'this', 'story', 'then'), ('this', 'story', 'then', 'he'), ('story', 'then', 'he', 'may'), ('then', 'he', 'may', 'have'), ('he', 'may', 'have', 'turned'), ('may', 'have', 'turned', 'out'), ('have', 'turned', 'out', 'to'), ('turned', 'out', 'to', 'be'), ('out', 'to', 'be', 'but'), ('to', 'be', 'but', 'this'), ('be', 'but', 'this', 'is'), ('but', 'this', 'is', 'just'), ('this', 'is', 'just', 'how'), ('is', 'just', 'how', 'i'), ('just', 'how', 'i', 'saw'), ('how', 'i', 'saw', 'it'), ('i', 'saw', 'it', 'in'), ('saw', 'i

Making quadgrams worked. Now I have to remove the stopwords.

In [53]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amysillman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
import re
import string

In [48]:
##This was something I was doing to see if I could remove stopwords from a list of a list of tuples.
##I was unable to apply the lambda function to either the list or the tuple.
tup_list=[[('and', 'Bear', 'Cat'), ('Apple', 'Bear', 'Cat')], [('and', 'Elephant', 'Frog'), ('Dog', 'Elephant', 'Frog')]]
tup_array=np.array(tup_list)
print(tup_array)
new_str_list=[]
for item in tup_array:
    new_list=[]
    for thing in item:
        #thing_str=' '.join(thing)
        thing_str_list=list(thing)
        #print(thing_str)
        print(thing_str_list)
        thing_str_list.apply(lambda x: [item for item in x if item not in stop])
        new_list.append(thing_str_list)
    new_str_list.append(new_list)
print(new_str_list)   

[[['and' 'Bear' 'Cat']
  ['Apple' 'Bear' 'Cat']]

 [['and' 'Elephant' 'Frog']
  ['Dog' 'Elephant' 'Frog']]]
['and', 'Bear', 'Cat']


AttributeError: 'list' object has no attribute 'apply'

In [40]:
#remove stopwords from the list of quadgrams
stop=stopwords.words('english')
katniss_quadgrams_nostop_list=[]
for story in katniss_quadgrams_list:
    new_story=[]
    for q_gram in story:
        q_gram_str=' '.join(q_gram)
        q_gram_str.apply(lambda x: [item for item in x if item not in stop])
        new_story.append(q_gram_str)
    katniss_quadgrams_nostop_list.append(new_story)
#katniss_quadgrams_nostop_list is the list of stories, where each story is a list of quadgrams in tuples

AttributeError: 'tuple' object has no attribute 'apply'

In [54]:
##Try removing stopwords from the dataframe column with the quadgrams
stop=stopwords.words('english')
katniss['quadgrams_nostops']=katniss['story_text_quadgrams'].apply(lambda x: [item for item in x if item not in stop])


In [56]:
#That didn't take out the stopwords. I'll drop the new column and try something else.
katniss.drop(['quadgrams_nostops'], axis=1, inplace=True)

In [57]:
katniss.head(1)

,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_wtokenized,story_text_quadgrams
0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...","[for, the, tumblr, everlark, fic, exchange, sp...","[(for, the, tumblr, everlark), (the, tumblr, e..."


In [58]:
##Try removing stopwords from the dataframe column with the quadgrams
stop=stopwords.words('english')
katniss['quadgrams_nostops']=(katniss['story_text_quadgrams']).apply(lambda x: [item for item in x if item not in stop])

In [59]:
katniss.head(1)

,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_wtokenized,story_text_quadgrams,quadgrams_nostops
0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...","[for, the, tumblr, everlark, fic, exchange, sp...","[(for, the, tumblr, everlark), (the, tumblr, e...","[(for, the, tumblr, everlark), (the, tumblr, e..."


In [60]:
#that didn't work. drop the column
katniss.drop(['quadgrams_nostops'], axis=1, inplace=True)

In [61]:
katniss.head(1)

,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_wtokenized,story_text_quadgrams
0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...","[for, the, tumblr, everlark, fic, exchange, sp...","[(for, the, tumblr, everlark), (the, tumblr, e..."


In [ ]:
stop=stopwords.words('english')
katniss_quadgrams_nostop_list=[]
for story in katniss['story_text_quadgrams']:
    new_story=[]
    for q_gram in story:
        for word in q_gram:
            #q_gram_str=' '.join(q_gram)
            word.apply(lambda x: [item for item in x if item not in stop])
        new_story.append(q_gram_str)
    katniss_quadgrams_nostop_list.append(new_story)

In [63]:
#katniss['quadgrams_nostops'] = katniss['story_text_quadgrams'].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in stop))

I don't know if I'm going to be able to remove stopwords from the quadgrams. 
I'll try doing it on the story_text tokenized by single words. See Notebook 3.